In [ ]:
from __future__ import print_function
from astropy.io import fits
import numpy as np
import math
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
from numpy.polynomial.polynomial import polyval, polyder
%matplotlib inline

In [ ]:
from AlgorithmDev import PolygonClipping

In [ ]:
def plot_fits_trace(spectral, total_rows, coeffs_rows):
    plt.figure(figsize=(12,12))
    plt.subplot(1, 1, 1)
    im = plt.imshow(spectral['data'], cmap='gray', norm=LogNorm())
    x_val = list(range(0, spectral['xdim']))
    for y in range(0, total_rows):
        y_val = [polyval(x, coeffs_rows[y]) for x in x_val]
        plt.plot(x_val, y_val, 'r--')
    plt.ylim(0, spectral['ydim'])
    plt.colorbar(im, fraction=0.046, pad=0.04)   

In [ ]:
def plot_output(out_data, total_rows):
    # show output
    plt.figure(figsize=(20,20))
    plt.subplot(1, 1, 1)
    plt.imshow(out_data, cmap='gray')
    plt.ylim(0, total_rows)
    #plt.colorbar(im, fraction=0.046, pad=0.04)

In [ ]:
def load_spectral_sample(fits_file, order_trace_csv):
    poly_c = PolygonClipping(fits_file, 6)
    spectral = poly_c.load_paras_spectral()
    print('data size: ', spectral['xdim'], spectral['ydim'])
    coeffs_rows = poly_c.load_csv_file(order_trace_csv)
   
    return {'spectral': spectral, 'coeffs': coeffs_rows, 'poly_handle': poly_c}

In [ ]:
def analyze_spectral(spectral, coeffs_rows, poly_c, row_idx=None):
    total_rows = len(coeffs_rows)
    in_data = np.array(spectral.get('data'), None)
    out_data = np.zeros((total_rows, spectral.get('xdim')))
    
    print('row: ')
    row_range = range(0, total_rows) if row_idx is None else range(row_idx, row_idx+1)
   
    for i in row_range:
        print(i, end=' ')
        result_data = poly_c.rectify_spectral_curve(coeffs_rows[i], in_data, verbose=False)
        line_data = result_data.get('out_data')
        #import pdb; pdb.set_trace()
        # do the sum internally
        #result_data = poly_c.rectify_spectral_curve(coeffs_rows[i], in_data, sum_extraction=False)
        #print('result_data:', result_data.get('out_data'))
        #line_data = poly_c.sum_curve(result_data.get('out_data'))
        poly_c.fill_2D_to_2D(line_data, out_data, 0, i)
        
    
    print('done')
    return {'out_data': out_data, 'dim':{'height': total_rows, 'width': spectral['xdim']}}
    

In [ ]:
def make_fits(data, output_fits):
    hdu = fits.PrimaryHDU(data)
    hdu.writeto(output_fits)

In [ ]:
fits_base = '../test_data/paras_data/a00'
csv_base = '../test_data/paras_data/order_trace_'
output_base = '../test_data/paras_data/results_sum/result_'
fits_list = ['18', '19', '20', '21']
csv_list = ['A', 'B']

In [ ]:
f = 0
c = 1
csv_name = csv_list[c]
fits_name = fits_list[f]
fits_file = fits_base+fits_name+'.fits'
csv_file = csv_base+csv_name+'.csv'
output_fits = output_base + csv_name + '_' + fits_name + '_sum.fits'
print('output file: ', output_fits)
sample_info = load_spectral_sample(fits_file, csv_file)

spectral = sample_info.get('spectral')
coeffs_rows = sample_info.get('coeffs')
poly_c = sample_info.get('poly_handle')
plot_fits_trace(spectral, len(coeffs_rows), coeffs_rows)

In [ ]:
result_data = analyze_spectral(spectral, coeffs_rows, poly_c)
#import pdb; pdb.set_trace()
plot_output(result_data.get('out_data'), result_data.get('dim').get('height'))

In [ ]:
make_fits(result_data.get('out_data'), output_fits)